In [1]:
%matplotlib inline
import codecs, os, sys, json, string
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
from sklearn import preprocessing
# import sympy
# from sympy import symbols, integrate
from sympy import *

In [2]:
def join_path(d, file):
    return os.path.join(d, file)

In [29]:
data_path = "..\\data\\extracted\\"
bhot100 = pd.read_csv(join_path(data_path, 'hot-100.csv'))
bhot100['title'] = bhot100.apply(lambda row: row['title'].lower().strip(), axis=1)
bhot100['artist_name'] = bhot100.apply(lambda row: row['artist_name'].lower().strip(), axis=1)
grammy = pd.read_csv(join_path(data_path, 'grammy.csv'), 
                     dtype={'award_for': 'str',
                            'name': 'str'
                        })
grammy['award_for'] = grammy.apply(lambda row: str(row['award_for']).lower().strip(), axis=1)
grammy['name'] = grammy.apply(lambda row: str(row['name']).lower().strip(), axis=1)
grammy = grammy[grammy['award_type']=='Individual']
ama = pd.read_csv(join_path(data_path, 'ama.csv'), 
                    dtype={'award_for': 'str',
                            'name': 'str'
                    })
ama['award_for'] = ama.apply(lambda row: str(row['award_for']).lower().strip(), axis=1)
ama['name'] = ama.apply(lambda row: str(row['name']).lower().strip(), axis=1)

msd = pd.read_csv(join_path(data_path, 'MSDSet.csv'))
msd['artist_name'] = msd.apply(lambda row: str(row['artist_name']).lower().strip(), axis=1)
msd['title'] = msd.apply(lambda row: str(row['title']).lower().strip(), axis=1)

In [12]:

titles = bhot100.title
merge_song_bhot100 = bhot100[['title', 'artist_name', 'current_rank']].groupby(['title', 'artist_name']).sum()
count_onchart_bhot100 = bhot100[['title', 'artist_name', 'current_rank']].groupby(['title', 'artist_name'])\
                        .count().rename(columns={"current_rank": "onchart_count"})
bhot100_song_num = len(merge_song_bhot100)

# count_onchart_bhot100 = count_onchart_bhot100.sort_values('onchart_count', ascending=False)
# count_onchart_bhot100
avr_bhot100_ranks = merge_song_bhot100.current_rank / count_onchart_bhot100.onchart_count
avr_bhot100_ranks.sort_values()
# avr_bhot100_ranks['Radioactive']


title                                            artist_name                                
zeze                                             kodak black featuring travis scott & offset      4.666667
in my feelings                                   drake                                            5.722222
sicko mode                                       travis scott                                     6.307692
girls like you                                   maroon 5 featuring cardi b                       6.478261
get back                                         the beatles with billy preston                   6.833333
mrs. brown you've got a lovely daughter          herman's hermits                                 7.000000
i want to hold your hand                         the beatles                                      7.533333
hey jude                                         the beatles                                      7.842105
i like it                                        ca

In [ ]:
overall_avr_bhot_rank = np.mean(avr_bhot100_ranks)
print (overall_avr_bhot_rank)
blamda = 1/overall_avr_bhot_rank
print (blamda)

In [ ]:
def rank_prob(rank):
    x = symbols('x')
    return integrate(blamda * exp(blamda*(x-100)), (x, rank-1, rank))
#     return blamda*np.exp(blamda*(rank-100))

In [ ]:
rank_prob(1)/rank_prob(2)

In [ ]:
rank_prob(1)/rank_prob(10)

In [ ]:
rank_prob(1)/rank_prob(25)

In [ ]:
rank_prob(1)/rank_prob(50)

In [ ]:
rank_prob(1)/rank_prob(100)

In [ ]:
rank_prob(1)

In [ ]:
rank_prob_table_bhot100 = [float(rank_prob(i)) for i in range(1, 101)]

In [ ]:
rank_prob_table_bhot100

In [ ]:
# f_score_table_bhot100 = -np.log(rank_prob_table_bhot100)
# f_score_table_bhot100
# f_norm_score_table_bhot100 = preprocessing.MinMaxScaler()\
#         .fit_transform(np.array(f_score_table_bhot100).reshape(-1, 1))
# f_norm_score_table_bhot100 = f_norm_score_table_bhot100.reshape(-1).tolist()
# f_norm_score_table_bhot100

In [ ]:
score_table_bhot100 = [1/i for i in rank_prob_table_bhot100]
score_table_bhot100

In [ ]:
norm_score_table_bhot100 = preprocessing.MinMaxScaler((1, 100))\
        .fit_transform(np.array(score_table_bhot100).reshape(-1, 1))
norm_score_table_bhot100 = (norm_score_table_bhot100).reshape(-1).tolist()
norm_score_table_bhot100

In [ ]:
grammy_num = len(grammy)
grammy['cnt'] = grammy['year']
artist_grammy_count = grammy[['name', 'cnt']].groupby(['name']).count()
artist_grammy_count

In [ ]:
# both grammy and billboard started from 1958
grammy_score = bhot100_song_num/grammy_num*100
grammy_score

In [ ]:
#ama started from 1974
ama['cnt'] = ama['year']
ama_num = len(ama)
ama_score = bhot100_song_num/ama_num*100*(2018-1974)/(2018-1958)
ama_score

In [ ]:
def generate_feature():
#     songs = set()
#     msdtitle = msd['title'].tolist()
#     msdartist = msd['artist_name'].tolist()
#     songsmap = merge_song_bhot100.reset_index()\
#         .apply(lambda row: row['title'] in msdtitle and row['artist_name'] in msdartist, axis=1)
#     songs = merge_song_bhot100.reset_index()[['title', 'artist']][songsmap]
    songs = pd.read_csv(join_path(data_path, 'bh100_inmsd_songs.csv'))
    print (len(songs))
    fd = {}
    for i in range(0, len(songs)):
        row = songs.iloc[i]
        t = row['title']
        an = row['artist_name']
        index = row['title'] + ' by ' + row['artist_name']
        if index not in fd:
            fd[index] = {
                'title': row['title'],
                'artist_name': row['artist_name'],
                'bb_rank': [],
                'grammy': [],
                'ama': [],
                
            }
        bills = bhot100[(bhot100['title']==t) & (bhot100['artist_name']==an)]
        lifespan = pd.DataFrame({'chart_date':bills['chart_date'], 'release_date': row['chart_date']})
        lifespan = pd.to_datetime()
        lifespan['chart_date']=pd.to_datetime(lifespan['chart_date'],format='%Y-%m-%d')
        lifespan['release_date']=pd.to_datetime(lifespan['release_date'],format='%Y-%m-%d')
        lifespan['dayspan']=(lifespan['last_time']-lifespan['first_time']).dt.days
        lifespan['weekspan']=lifespan['dayspan']/7

In [30]:
msdtitle = msd['title'].tolist()
msdartist = msd['artist_name'].tolist()
songsmap = merge_song_bhot100.reset_index()\
    .apply(lambda row: (row['title'] in msdtitle) and (row['artist_name'] in msdartist), axis=1)
print(len(songsmap))

27843


In [32]:
songs = merge_song_bhot100.reset_index()[['title', 'artist_name']][songsmap]
print (len(songs))
songs.to_csv(join_path(data_path, 'bh100_inmsd_songs.csv'))
# songs['release_date'] = songs.apply(lambda row: msd[(msd['title']==row['title']) & \
#                                                     (msd['artist_name']==row['artist_name'])]['release_']

13804


In [27]:
for i in range(8000, 8010):
    print (merge_song_bhot100.reset_index()['title'].iloc[i], merge_song_bhot100.reset_index()['artist_name'].iloc[i])

gonna wanna tonight chase rice
gonna' get along without you now tracey dey
gonorrhea lil wayne featuring drake
gonzo james booker
goo goo barabajagal (love is hot) donovan/jeff beck group
good better than ezra
good clean fun the monkees
good combination sonny & cher
good day sunshine claudine longet
good directions billy currington


,file_id,song_id,post_year,artist_id,artist_hotness,artist_name,danceability,duration,end_of_fade_in,energy,loudness,song_hotness,start_of_fade_out,tempo,title
0,A/E/E/TRAEEGO12903CF7D27,SOGXWRE12AC468BE24,1966,ARTXS851187FB411C7,0.338575,Chris Farlowe,0.0,177.57995,0.346,0.0,-11.027,NaN,175.078,146.913,Paint It Black
1,A/E/E/TRAEEIJ128F92DFEDB,SONEGRP12A58A7F9DB,1997,ARXGQ2X1187FB4B346,0.389039,Charles Brown,0.0,167.81016,0.136,0.0,-17.772,0.262309,162.946,142.058,More Than You Know
2,A/E/E/TRAEEMK12903CF5AD5,SOEMNOG12AC960A5E3,1995,ARY6PB61187FB45550,0.373021,Dub War,0.0,236.51220,0.984,0.0,-7.632,0.000000,231.178,105.845,Strike It (Vpro Radio Session)
3,A/E/E/TRAEEVW128F4244269,SOJJUPJ12A6D4FC655,0,AR0CJQS1187FB42BEB,0.273445,Big Maceo Merriweather,0.0,175.85587,0.566,0.0,-15.873,0.482867,169.755,169.170,Chicago Breakdown
4,A/E/E/TRAEEDH128F932BF7D,SOBGCUO12A6D4FB2B0,2006,AR5CKT41187B9B9AB0,0.436170,Presuntos Implicados,0.0,249.80853,0.000,0.0,-8.518,NaN,240.013,116.924,Mil Mariposas
